# LSTM Regression Model

In [1]:
from deep_river.regression import RollingRegressor
from river import (
    metrics,
    compose,
    preprocessing,
    datasets,
    stats,
    feature_extraction,
)
from torch import nn
from tqdm import tqdm

In [2]:
def get_hour(x):
    x["hour"] = x["moment"].hour
    return x

## Simple RNN Regression Model

In [3]:
class RnnModule(nn.Module):

    def __init__(self, n_features, hidden_size):
        super().__init__()
        self.n_features = n_features
        self.rnn = nn.RNN(
            input_size=n_features, hidden_size=hidden_size, num_layers=1
        )
        self.fc = nn.Linear(in_features=hidden_size, out_features=1)

    def forward(self, X, **kwargs):
        output, hn = self.rnn(X)  # lstm with input, hidden, and internal state
        return self.fc(output[-1, :])

In [4]:
dataset = datasets.Bikes()
metric = metrics.MAE()

model_pipeline = compose.Select(
    "clouds", "humidity", "pressure", "temperature", "wind"
)
model_pipeline += get_hour | feature_extraction.TargetAgg(
    by=["station", "hour"], how=stats.Mean()
)
model_pipeline |= preprocessing.StandardScaler()
model_pipeline |= RollingRegressor(
    module=RnnModule,
    loss_fn="mse",
    optimizer_fn="sgd",
    window_size=20,
    lr=1e-2,
    hidden_size=32,  # parameters of MyModule can be overwritten
    append_predict=True,
)
model_pipeline

Pipeline (
  TransformerUnion (
    Select (
      clouds
      humidity
      pressure
      temperature
      wind
    ),
    Pipeline (
      FuncTransformer (
        func="get_hour"
      ),
      TargetAgg (
        by=['station', 'hour']
        how=Mean ()
        target_name="y"
      )
    )
  ),
  StandardScaler (
    with_std=True
  ),
  RollingRegressor (
    module=None
    loss_fn="mse"
    optimizer_fn="sgd"
    lr=0.01
    is_feature_incremental=False
    window_size=20
    append_predict=True
    device="cpu"
    seed=42
  )
)

In [5]:
for x, y in tqdm(dataset.take(5000)):
    y_pred = model_pipeline.predict_one(x)
    metric.update(y_true=y, y_pred=y_pred)
    model_pipeline.learn_one(x=x, y=y)
print(f"MAE: {metric.get():.2f}")

0it [00:00, ?it/s]

1it [00:00,  1.34it/s]

39it [00:00, 62.13it/s]

95it [00:00, 154.88it/s]

158it [00:01, 254.11it/s]

...

4847it [00:09, 583.84it/s]

4906it [00:09, 571.39it/s]

4964it [00:09, 557.54it/s]

5000it [00:09, 528.93it/s]

MAE: 3.79


## LSTM Regression Model

In [6]:
class LstmModule(nn.Module):

    def __init__(self, n_features, hidden_size=1):
        super().__init__()
        self.n_features = n_features
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(
            input_size=n_features,
            hidden_size=hidden_size,
            num_layers=1,
            bidirectional=False,
        )
        self.fc = nn.Linear(in_features=hidden_size, out_features=1)

    def forward(self, X, **kwargs):
        output, (hn, cn) = self.lstm(
            X
        )  # lstm with input, hidden, and internal state
        return self.fc(output[-1, :])

In [7]:
dataset = datasets.Bikes()
metric = metrics.MAE()

model_pipeline = compose.Select(
    "clouds", "humidity", "pressure", "temperature", "wind"
)
model_pipeline += get_hour | feature_extraction.TargetAgg(
    by=["station", "hour"], how=stats.Mean()
)
model_pipeline |= preprocessing.StandardScaler()
model_pipeline |= RollingRegressor(
    module=LstmModule,
    loss_fn="mse",
    optimizer_fn="sgd",
    window_size=20,
    lr=1e-2,
    hidden_size=32,  # parameters of MyModule can be overwritten
    append_predict=True,
)
model_pipeline

Pipeline (
  TransformerUnion (
    Select (
      clouds
      humidity
      pressure
      temperature
      wind
    ),
    Pipeline (
      FuncTransformer (
        func="get_hour"
      ),
      TargetAgg (
        by=['station', 'hour']
        how=Mean ()
        target_name="y"
      )
    )
  ),
  StandardScaler (
    with_std=True
  ),
  RollingRegressor (
    module=None
    loss_fn="mse"
    optimizer_fn="sgd"
    lr=0.01
    is_feature_incremental=False
    window_size=20
    append_predict=True
    device="cpu"
    seed=42
  )
)

In [8]:
for x, y in tqdm(dataset.take(5000)):
    y_pred = model_pipeline.predict_one(x)
    metric.update(y_true=y, y_pred=y_pred)
    model_pipeline.learn_one(x=x, y=y)
print(f"MAE: {metric.get():.2f}")

0it [00:00, ?it/s]

53it [00:00, 525.49it/s]

113it [00:00, 563.58it/s]

175it [00:00, 585.50it/s]

238it [00:00, 600.30it/s]

...

4873it [00:07, 629.56it/s]

4936it [00:07, 628.20it/s]

4999it [00:08, 623.88it/s]

5000it [00:08, 621.44it/s]

MAE: 2.78
